In [1]:
!pip install langchain-community==0.0.11 pypdf==3.17.4 langchain==0.1.0 python-dotenv==1.0.0 langchain-openai==0.0.2.post1 faiss-cpu==1.7.4 tiktoken==0.5.2 langchainhub==0.1.14 pip sentence-transformers


[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: pip install --upgrade pip


In [2]:
from dotenv import load_dotenv
from langchain import hub
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_community.vectorstores.faiss import FAISS
from langchain_community.llms import Ollama
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

pdf_path = "pdf/red_hat_openshift_ai_self-managed-2-latest-working_on_data_science_projects-en-us.pdf"

def main():
  load_dotenv()
  loader = PyPDFLoader(file_path=pdf_path)
  documents = loader.load()
  text_splitter = CharacterTextSplitter( chunk_size=1000, chunk_overlap=50, separator="\n" )
#  docs = text_splitter.split_documents(documents)
  embeddings = HuggingFaceEmbeddings()
#  vectorstore = FAISS.from_documents(docs, embeddings)
#  vectorstore.save_local("vector_db")

  retrieval_qa_chat_prompt = hub.pull("langchain-ai/retrieval-qa-chat")

  #llm = ChatOpenAI()
    
  #llm = Ollama(model="mistralai/Mistral-7B-Instruct-v0.2", base_url="http://ollama.redis.svc.cluster.local:11434")
  llm = Ollama(
    base_url="http://ollama.redis.svc.cluster.local:11434",
    model="mistral",
#    top_p=0.92,
#    temperature=0.01,
#    num_predict=512,
#    repeat_penalty=1.03,
    callbacks=[StreamingStdOutCallbackHandler()])  

  combine_docs_chain = create_stuff_documents_chain(llm, retrieval_qa_chat_prompt)

  retriever = FAISS.load_local("vector_db", embeddings).as_retriever()
  retrieval_chain = create_retrieval_chain(retriever, combine_docs_chain)

  response = retrieval_chain.invoke(
    {"input": "What is openshift AI?"}
  )

  print(response["answer"])


if __name__ == "__main__":
  main()

 OpenShift AI is a platform from Red Hat that allows data scientists to organize their data science work into projects. It provides features like workbenches (which can contain Jupyter notebooks), cluster storage, data connections, pipelines, and models with model servers. These components help in creating and collaborating on notebooks, training and deploying models, configuring model servers, and implementing pipelines for standardizing and automating machine learning workflows. It's important to note that if you create an OpenShift project outside of the OpenShift AI user interface, it won't be displayed as a data science project. OpenShift AI is a platform from Red Hat that allows data scientists to organize their data science work into projects. It provides features like workbenches (which can contain Jupyter notebooks), cluster storage, data connections, pipelines, and models with model servers. These components help in creating and collaborating on notebooks, training and deploy